# Basics

of return calculations: returns, average returns and volatilities of portfolios.

## Load Packages and Extra Functions

The notebook uses the functions `printmat()` and `printlnPs()` for formatted printing of matrices and numbers. These functions are in the included `src/printmat.jl` file and call on the `Printf` package. 

Also, the `lag()` function (from `src/lag.jl`) lags a vector/matrix.

In [1]:
using Printf

include("src/printmat.jl")
include("src/lag.jl");       #; to suppress printing of last command in cell

# Return Calculations

The return of holding the asset between $t-1$ and $t$ is

$R_t = (P_t+D_t)/P_{t-1} - 1,$

where $P_t$ is the price (measured after dividends) and $D_t$ is the dividend.

We can calculate the returns by a loop or by a more compact notation, see below.

### A Remark on the Code
For the vectorized version, notice that
- `lag(P)` creates a vector `[NaN,P[1],...,P[end-1]`, that is, the previous (lagged) value of `P`.
- we use `[a,b]./[c,d]` to do element-by-element division. Also, use `[a,b] .- 1` to subtract 1 from each element. In contrast, `[a,b]-[c,d]` (or `+`) needs no dot since that is a traditional mathematical operation.

In [2]:
P = [100,108,109]                     #prices (after dividends) for t=1,2,3
D = [0,2,0]                           #dividends, could also use [0;2;0]

R = zeros(length(P))                  #where to store the results
for t in 2:length(P)                  #P[2] is the 2nd element of P  
    R[t] = (P[t] + D[t])/P[t-1] - 1
end
R = R[2:end]                          #get rid of R[1] since we have no return there

R_alt = (P + D)./lag(P) .- 1          #vectorized alternative, notice the ./ and .- 1

printmat(R*100,R_alt[2:end]*100,colNames=["return, %","return (alt), %"],rowNames=2:3,cell00="period",width=15)

period      return, %return (alt), %
2              10.000         10.000
3               0.926          0.926



# Excess Returns

Subtract a reference return, often a safe return, to get an excess return

$R^e_t = R_t - R_{ft}$

In [3]:
Rf = 0.01

Rᵉ = R .- Rf        #R\^e [TAB] to get Rᵉ
printmat(Rᵉ)

     0.090
    -0.001



# Descriptive Statistics

We need some extra packages for reading in data. 

Data is monthly so we annualize means by multiplying by 12 and standard deviations by $\sqrt{12}$.

In [4]:
using Statistics, DelimitedFiles

In [5]:
x = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)
ym = x[:,1]
x  = x[:,2:4];    #market, small minus big, high minus low

println("Sample period: ",ym[1],"-",ym[end])

Sample period: 197901.0-201104.0


In [6]:
μ  = mean(x,dims=1)*12
σ  = std(x,dims=1)*sqrt(12)
SR = μ./σ

xx = vcat(μ,σ,SR)             #to print
printmat(xx;rowNames=["mean","std","SR"],colNames=["Rme","SMB","HML"])

           Rme       SMB       HML
mean     7.223     2.500     3.960
std     15.949    10.852    10.831
SR       0.453     0.230     0.366



# Cumulating Returns

Net returns can be cumulated to calculate portfolio values as 

$V_t = V_{t-1}(1+R_t)$

where we need a starting value (initial investment) for the portfolio (a common choice is to normalise to $V_0=1$).

With log returns, $r_t=\log(1+R_t)$, we instead do 

$\ln V_t = \ln V_{t-1} + r_t$

If the return series is an excess return, add the riskfree rate to convert it to get net returns - and then cumulate as described above.

### A Remark on the Code
- Use `cumprod([a,b]` to calculate `[a,a*b]` and `cumsum([a,b]` to calculate `[a,a+b]`.
- To add 1 to each element of an array `R`, do `1 .+ R` (Notice the dot and the space before the dot.)
- To calculate the logarithm of each value in a matrix `X`, do `log.(X)` Again, notice the dot. In general, a function `fn(x)` that is defined for a scalar can be called with a dot `fn.(X)` to do the calculation for each element in an array (vector, matrix,...).

In [7]:
R   = [20,-35,25]/100                #returns for t=1,2,3
V   = cumprod(1 .+ R)                #V(t) = V(t-1)*(1+R(t)), starting at 1 in t=0
r   = log.(1 .+ R)                   #log returns
lnV = cumsum(r)                      #lnV(t) = lnV(t-1) + r(t) 

printmat(R,V,lnV,colNames=["R","V","lnV"],rowNames=1:3,cell00="period")

printred("Check that lnV really equals log.(V). Also, try a loop instead")

period         R         V       lnV
1          0.200     1.200     0.182
2         -0.350     0.780    -0.248
3          0.250     0.975    -0.025

Check that lnV really equals log.(V). Also, try a loop instead


# Portfolio Return

We form a portfolio by combining $n$ assets: $v$ is the vector of $n$ portfolio weights. The portfolio return is

$R_v = v'R,$

where $R$ is a vector of returns of the $n$ assets.

In [8]:
v = [0.8,0.2]
R = [10,5]/100          #returns of asset 1 and 2
Rᵥ = v'R                #R\_v[TAB] to get Rᵥ

printblue("Portfolio weights:")
printmat(v;rowNames=["asset 1","asset 2"])

printblue("Returns:")
printmat(R;rowNames=["asset 1","asset 2"])

printblue("Portfolio return: ")
printlnPs(Rᵥ)

Portfolio weights:
asset 1     0.800
asset 2     0.200

Returns:
asset 1     0.100
asset 2     0.050

Portfolio return: 
     0.090


## Portfolio Return with a Riskfree Asset

The previous expression for a portfolio return is still correct when one of the returns (in $R$) is riskfree. However, we will often use an an alternative form

$R_p = v'R + (1-1'v)R_f$,

or (equivalently) as 

$R_p = v'R^e + R_f$,

where $v$ now are the weights on the $n$ risky assets and no longer required to sum to 1.

In Julia `1'v` can be calculated as `ones(n)'v` or (perhaps easier) `sum(v)`.

In [9]:
Rf = 0.01

Rᵉ = R .- Rf

v = [0.55,0.2]

printlnPs("weight on riskfree asset: ",1-sum(v))

Rₚ = v'R + (1-sum(v))*Rf
Rₚ_alt = v'Rᵉ + Rf

printlnPs("Portfolio return, two versions: ",Rₚ," and  ",Rₚ_alt)

weight on riskfree asset:      0.250
Portfolio return, two versions:      0.068     and       0.068
